# Test controlling system using LP implementation

In [1]:
import os
os.chdir('..') # move to the root directory (from dev)

In [2]:
import numpy as np
import time
from tqdm.notebook import tqdm
from utils import build_schema
from linmodel import LinProgModel
from citylearn.citylearn import CityLearnEnv

In [3]:
# set up parameters
dataset_dir = os.path.join('data','processed') # dataset directory

base_params = {
    'data_dir_path': os.path.join('data','processed'),
    'weather_data_path': 'weather.csv',
    'carbon_intensity_data_path': 'carbon_intensity.csv',
    'pricing_data_path': 'pricing.csv',
    'schema_name': 'schema_build_test',
    'battery_efficiencies': [0.95,0.95,0.95,0.95],
    'battery_energy_capacities': None,
    'battery_power_capacities': None,
    'pv_power_capacities': None
}

In [4]:
# set up LP costs
cost_dict = {
    'carbon': 1.0, #5e-1,
    'battery': 1e3, #1e3,
    'solar': 1e3, #2e3,
    'grid_capacity': 5e-2*365/0.95,
    'grid_excess': 10e-2*365/0.95,
    'opex_factor': 20,
    'battery_power_ratio': 0.4
}

In [5]:
# set up load profiles
building_years = [(40,2015),(4,2012),(40,2016),(4,2013)]

In [6]:
# build schema
params = base_params.copy()
params['building_names'] = [f'TB{i}' for i in range(len(building_years))]
params['load_data_paths'] = [f'ly_{b}-{y}.csv' for b,y in building_years]
params['schema_name'] = f'SP_test_schema'
schema_path = build_schema(**params)

In [7]:
tau = 48

In [8]:
env = CityLearnEnv(schema=schema_path)

In [9]:
# Initialise Linear MPC object.
lp = LinProgModel(env=env)
lp.tau = tau
lp.generate_LP(cost_dict,design=False,grid_capacity=400)

In [10]:
# Initialise control loop.
lp_solver_time_elapsed = 0
num_steps = 0
done = False

In [11]:
# Initialise environment.
observations = env.reset()
soc_obs_index = 22
current_socs = np.array([[charge*capacity for charge,capacity in zip(np.array(observations)[:,soc_obs_index],lp.battery_capacities.flatten())]]) # get initial SoCs

In [12]:
# Execute control loop.
with tqdm(total=env.time_steps) as pbar:

    while not done:
        if num_steps%100 == 0:
            pbar.update(100)

        # Compute MPC action.
        # ====================================================================
        if (num_steps <= (env.time_steps - 1) - tau):
            # setup and solve predictive Linear Program model of system
            lp_start = time.perf_counter()
            lp.set_time_data_from_envs(t_start=num_steps, tau=tau, initial_socs=current_socs) # load ground truth data
            lp.set_LP_parameters()
            results = lp.solve_LP(ignore_dpp=False)
            actions: np.array = results['battery_inflows'][0][:,0].reshape((lp.N,1))/lp.battery_capacities
            lp_solver_time_elapsed += time.perf_counter() - lp_start

        else: # if not enough time left to grab a full length ground truth forecast: do nothing
            actions = np.zeros((lp.N,1))

        # Apply action to environment.
        # ====================================================================
        observations, _, done, _ = env.step(actions)

        # Update battery states-of-charge
        # ====================================================================
        current_socs = np.array([[charge*capacity for charge,capacity in zip(np.array(observations)[:,soc_obs_index],lp.battery_capacities.flatten())]])

        num_steps += 1

  0%|          | 0/8760 [00:00<?, ?it/s]

/Users/mal84/opt/miniconda3/envs/BD-VOI/lib/python3.8/site-packages/cvxpy/reductions/solvers/conic_solvers/scipy_conif.py:99: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (StrictVersion(scipy.__version__) < StrictVersion('1.6.1')):


In [13]:
for b in env.buildings:
    print(b.electrical_storage.soc)

[0, 0.0, 0.0, 13.708433601405682, 13.708433601405682, 13.708433601405682, 13.708433601405682, 13.708433601405682, 32.633387332493626, 47.042360967072874, 86.02944426151193, 99.99955867758806, 99.99948701209695, 99.99948701174769, 99.99948701174769, 99.99948701174769, 82.07836010055968, 41.039472385527894, 0.0006667487642744163, 6.6696577070935945e-09, 6.6696577070935945e-09, 6.671850149640394e-14, 6.674043313131226e-19, 6.674043313131226e-19, 38.986787379303834, 0.0004679696055021054, 0.0004679696055021054, 22.025339019581907, 61.011930299280415, 99.99829668640629, 61.01356463629778, 99.99790669546091, 99.99770670581941, 99.99770670484476, 99.99770670484476, 99.99770670484476, 61.013716659222204, 61.01371665922217, 61.01371665922217, 99.9975167160209, 58.95948382405271, 17.921533010352697, 0.0005525695411918718, 1.7519827165629956e-08, 5.554859284269965e-13, 1.7612309400773646e-17, 1.7612309400773646e-17, 1.7612309400773646e-17, 1.7612309400773646e-17, 1.7612309400773646e-17, 1.7612309